In [1]:
%load_ext Cython

In [7]:
import networkx as nx
import numpy as np
from operator import itemgetter
def construct():
    g = nx.read_edgelist('BA500.csv',delimiter=",")
#     g = nx.barabasi_albert_graph(n=500, m=10,seed=2017)
    adjlist = nx.generate_adjlist(g, delimiter=",")
    adjlist = [np.array([eval(x)]).flatten() for x in adjlist]

    lens = [len(x) for x in adjlist]
    n = g.number_of_nodes()
    
    adjlist_m = np.ones([n, max(lens)], dtype = np.int32) * (-1)
    for nn in range(n):
        adjlist_m[nn, :lens[nn]] = adjlist[nn]
    
#     degree_list = [ [node,degree] for node, degree in g.degree(range(n)).items() ]
    degree_list = []
    for i in range(n):
        degree_list.append([i,sum(np.where(adjlist_m[i] != -1, 1, 0))-1])
    print(degree_list)
    sorted_degrees = sorted(degree_list, key=itemgetter(1), reverse = True)
    return sorted_degrees,adjlist_m

sorted_degrees,adjlist_m = construct()

[[0, 56], [1, 1], [2, 53], [3, 1], [4, 37], [5, 1], [6, 44], [7, 1], [8, 14], [9, 1], [10, 25], [11, 1], [12, 1], [13, 1], [14, 25], [15, 2], [16, 4], [17, 21], [18, 1], [19, 11], [20, 2], [21, 5], [22, 17], [23, 1], [24, 5], [25, 1], [26, 2], [27, 1], [28, 1], [29, 0], [30, 1], [31, 1], [32, 1], [33, 1], [34, 2], [35, 1], [36, 1], [37, 1], [38, 13], [39, 2], [40, 1], [41, 5], [42, 1], [43, 0], [44, 1], [45, 3], [46, 3], [47, 0], [48, 4], [49, 5], [50, 1], [51, 3], [52, 1], [53, 4], [54, 0], [55, 0], [56, 0], [57, 13], [58, 7], [59, 1], [60, 0], [61, 80], [62, 1], [63, 0], [64, 0], [65, 5], [66, 1], [67, 0], [68, 0], [69, 0], [70, 0], [71, 0], [72, 0], [73, 0], [74, 0], [75, 0], [76, 0], [77, 0], [78, 0], [79, 0], [80, 0], [81, 0], [82, 0], [83, 0], [84, 0], [85, 0], [86, 0], [87, 0], [88, 0], [89, 0], [90, 0], [91, 0], [92, 0], [93, 0], [94, 0], [95, 0], [96, 0], [97, 0], [98, 0], [99, 0], [100, 0], [101, 0], [102, 0], [103, 0], [104, 0], [105, 0], [106, 0], [107, 0], [108, 0], [109, 

In [18]:
%%cython 
import numpy as np
cimport numpy as np

'''
Ignorant: 0
Spreader: 1
Stifler : 2
'''

cdef _info_spread(np.ndarray[np.int32_t, ndim=2] adjlist_m, \
                np.ndarray[np.int32_t, ndim=1] states):  
    cdef int N = adjlist_m.shape[0]
    cdef int change = 0
    cdef float alpha = 0.4
    cdef float p = 0.2
    cdef float lamda = 0.2
    cdef Py_ssize_t n_i,n_j
    cdef np.ndarray[np.int32_t, ndim=1] new_states = states
    
    for n_i in range(N):
        # the node has been removed
        if adjlist_m[n_i,0] == -1:
            continue
        
        # n_i is not 'S'
        if states[n_i] == 2 or states[n_i] == 0:
            continue 
            
        # n_i is 'S'
        for n_j in adjlist_m[n_i,1:]: 
            # !! Discard Padding
            if n_j == -1:
                continue
            # n_j is 'I'
            if states[n_j] == 0:
                # I-->S
                if np.random.random() < alpha*p:
                    new_states[n_j] = 1
                    change += 1
                # I-->R
                elif np.random.random() < alpha*(1-p):
                    new_states[n_j] = 2
                    change += 1
            # n_j is 'R' or 'S'
            elif np.random.random() < lamda:
                    # S -> R
                    new_states[n_i] = 2
                    change += 1
                    break
            
    return new_states,change


def InfoSpread(adjlist, seeds, nodes_to_rm):
    N = adjlist.shape[0]
    change = 1 
    states = np.zeros(N,dtype=np.int32)
    
    # =================Initialize================
    '''
    rm nodes and set seeds --- Note: Don't set removed nodes as seeds
    '''
    # rm nodes from adjlist <=> set to -1
    for i in nodes_to_rm:
        for nns in adjlist[i,1:]:
            # disconnect i from its nns
            adjlist[nns,:] = np.where(adjlist[nns,:]==i, -1, adjlist[nns,:])
        # disconnect i and its nns
        adjlist[i,:] = -1

    #set seeds
    for seed in seeds:
        states[seed] = 1
    #rm nodes
    for i in nodes_to_rm:
        states[i] = -1
        
    
    #==============iter til converge==============
    while change:
        states,change = _info_spread(adjlist, states)
        
    return states

In [8]:
import matplotlib.pyplot as plt

# np.random.seed(2017)

def main_simulate(adjlist, nodes_to_rm):
    N = adjlist.shape[0]
    rm_nodes = []
    Avr_rhos = []
    for i in nodes_to_rm:
        rm_nodes.append(i)
        rho = 0
        for seed in range(N):
            final_states = InfoSpread(adjlist, [seed], rm_nodes)
            rho += sum(np.where(np.array(final_states) == 2, 1, 0))/N
        rho=rho/500
        
#         for seed in range(N):
#             for _ in range(100):
#                 final_states = InfoSpread(adjlist, [seed], rm_nodes)
#                 rho += sum(np.where(np.array(final_states) == 2, 1, 0))/N
#         rho=rho/500/100
        
        Avr_rhos.append(rho)
        
    plt.plot(range(25), Avr_rhos,'ro')
    plt.show()

if __name__ == '__main__':
    attack_list_k = np.array(sorted_degrees)[:25,0]
    main_simulate(adjlist_m,attack_list_k)

IndexError: too many indices for array

In [19]:
sorted_degrees

[]